In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files

In [3]:
# get kaggle api key from kaggle-->Account-->API section and upload it here.

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"teddwebber","key":"cbb9f79ca52e65c87cfcb88aaea2b210"}'}

In [4]:
# Creates hidden folder in root directory and saves kaggle.json in it.

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d ealaxi/paysim1
!unzip /content/paysim1.zip

100% 177M/178M [00:01<00:00, 89.5MB/s]
100% 178M/178M [00:01<00:00, 102MB/s] 
Archive:  /content/paysim1.zip
  inflating: PS_20174392719_1491204439457_log.csv  


In [20]:
import pandas as pd
import numpy as np

df = pd.read_csv("PS_20174392719_1491204439457_log.csv")
pd.set_option("display.max.columns", None)
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [21]:
len_not_fraud = len(df['isFraud'][df.isFraud == 0])
len_is_fraud = len(df['isFraud'][df.isFraud == 1])

arr = np.array([len_not_fraud, len_is_fraud])
labels = ['Not Fraudulent', 'Fraudulent']
print(f"Total No. of Non-Fraudulent Cases: {len_not_fraud}")
print(f"Total No. Fraudulent Cases: {len_is_fraud}")

Total No. of Non-Fraudulent Cases: 6354407
Total No. Fraudulent Cases: 8213


In [22]:
df.drop(['nameOrig','nameDest'], axis = 1, inplace = True)
data = df.copy(deep=True)
catCols = [col for col in data.columns if data[col].dtype=="O"]

from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

for item in catCols:
  data[item] = lb_make.fit_transform(data[item])

In [23]:
data

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,3,9839.64,170136.00,160296.36,0.00,0.00,0,0
1,1,3,1864.28,21249.00,19384.72,0.00,0.00,0,0
2,1,4,181.00,181.00,0.00,0.00,0.00,1,0
3,1,1,181.00,181.00,0.00,21182.00,0.00,1,0
4,1,3,11668.14,41554.00,29885.86,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...
6362615,743,1,339682.13,339682.13,0.00,0.00,339682.13,1,0
6362616,743,4,6311409.28,6311409.28,0.00,0.00,0.00,1,0
6362617,743,1,6311409.28,6311409.28,0.00,68488.84,6379898.11,1,0
6362618,743,4,850002.52,850002.52,0.00,0.00,0.00,1,0


In [24]:
# The function below will be used to evaluate different metrics of the algorithms used here.
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(y_test, y_pred):
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))
    print("Precision Score: ", precision_score(y_test, y_pred))
    print("Recall Score: ", recall_score(y_test, y_pred))
    print("F1 Score: ", f1_score(y_test, y_pred))
    print("Confusion Matrix: ", confusion_matrix(y_test, y_pred))

Logistic Regression

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 

X = data.drop('isFraud', axis=1)
y = data.isFraud


# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

lr = LogisticRegression()
model1 = lr.fit(X_train, y_train)
 
# Predict on training set
lr_pred = model1.predict(X_test)
evaluate_model(y_test, lr_pred)

Accuracy Score:  0.9983421923672953
Precision Score:  0.3843906510851419
Recall Score:  0.442150744119059
F1 Score:  0.4112525117213664
Confusion Matrix:  [[1587097    1475]
 [   1162     921]]


Random Forest Classifier

In [26]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

# predict on test set
rfc_pred = rfc.predict(X_test)

evaluate_model(y_test, rfc_pred)

Accuracy Score:  0.9996862927536141
Precision Score:  0.9788391777509069
Recall Score:  0.7772443590974556
F1 Score:  0.8664704308268664
Confusion Matrix:  [[1588537      35]
 [    464    1619]]


In [27]:
import joblib
joblib.dump(rfc, "Credit_fraud.pkl")

['Credit_fraud.pkl']